In [12]:
import pandas as pd
import sklearn
import numpy as np

data = pd.read_excel('../Adatok/tisztitott_adat_nonconcat.xlsx', header=0,index_col=0)
data

,date,text,president,label
0,2012-11-05,president obama tells the story of fired up re...,Obama,1
1,2012-11-06,election day is here confirm your polling plac...,Obama,1
2,2012-11-06,it’s election day this is your last chance to ...,Obama,1
3,2012-11-06,at the final rally of his final campaign last ...,Obama,1
4,2012-11-06,25 reasons that 25 people are voting for presi...,Obama,1
...,...,...,...,...
12104,2020-06-16,true …,Trump,-1
12105,2020-06-16,a great woman her son is looking down from hea...,Trump,-1
12106,2020-06-16,96% approval rating in the republican party th...,Trump,-1
12107,2020-06-17,joe biden was a total failure in government he...,Trump,-1


In [58]:
import gensim
tokenized_tweet = data['text'].apply(lambda x: x.split())

model_w2v = gensim.models.Word2Vec(
            tokenized_tweet,
            size=50, 
            window=5, 
            min_count=2,                                   
            sg = 1,
            hs = 0,
            negative = 10, 
            workers= 32, 
            seed = 34
)

model_w2v.train(tokenized_tweet, total_examples= len(data['text']), epochs=20)

(4272735, 5603020)

In [59]:
model_w2v.wv.most_similar(positive="trump")

[('donald', 0.8830842971801758),
 ('j', 0.7494584321975708),
 ('frame', 0.7408056259155273),
 ('takedown', 0.6770744323730469),
 ('clinton', 0.6687959432601929),
 ('web', 0.6605334877967834),
 ('alleged', 0.6578599214553833),
 ('derangement', 0.6541644334793091),
 ('russians', 0.6494784951210022),
 ('colluded', 0.6414963006973267)]

In [60]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model_w2v[word].reshape((1, size))
            count += 1.
        except KeyError:  # handling the case where the token is not in vocabulary
            continue
    if count != 0:
        vec /= count
    return vec

In [61]:
wordvec_arrays = np.zeros((len(tokenized_tweet), 50)) 
for i in range(len(tokenized_tweet)):
    wordvec_arrays[i,:] = word_vector(tokenized_tweet[i], 50)
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

(12109, 50)

In [64]:
vec_data = pd.concat([wordvec_df, data['label']], axis=1)
vec_data

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,label
0,-0.095463,0.139151,-0.356881,0.091876,0.220736,0.448557,0.328842,0.331583,0.124128,0.283225,...,0.023529,-0.111922,-0.233846,-0.151305,0.192978,0.547401,0.419040,-0.049735,-0.486088,1
1,0.015132,0.242092,-0.425064,-0.013810,0.167505,0.476959,0.281815,0.286692,0.018405,0.239796,...,-0.164217,-0.072055,-0.138283,-0.128062,0.281515,0.612355,0.338310,-0.153471,-0.434059,1
2,-0.125965,0.355396,-0.412121,0.078874,0.271823,0.535803,0.256289,0.379616,0.030027,0.075374,...,-0.138914,0.004187,-0.230320,-0.111103,0.345378,0.774623,0.320682,-0.135074,-0.384836,1
3,-0.110323,0.147059,-0.403875,0.004709,0.215338,0.471057,0.335582,0.341243,0.075930,0.255105,...,0.013237,-0.052945,-0.256318,-0.137915,0.273358,0.531114,0.476591,-0.042361,-0.461569,1
4,-0.162498,0.057550,-0.034758,-0.266167,0.264907,0.322853,0.422158,0.306710,-0.073539,0.108297,...,0.109074,0.077723,-0.388956,-0.319276,-0.062973,0.512198,0.355353,-0.367513,-0.374694,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12104,-0.194127,0.328952,-0.638252,-0.238019,0.055882,0.103523,0.786820,-0.166844,0.300118,0.410730,...,0.220658,0.226717,0.112656,0.154383,0.460603,0.776346,0.146590,0.019042,-0.080895,-1
12105,0.052348,0.085994,-0.105321,-0.145499,0.013745,0.398217,0.478521,0.338471,0.041541,0.113666,...,-0.262386,-0.047446,-0.002156,-0.225241,0.072200,0.634083,0.307751,0.039613,-0.218520,-1
12106,-0.179964,0.066921,-0.333907,-0.160486,0.172650,0.119559,0.356862,0.629339,0.248059,0.356602,...,-0.107896,0.268739,-0.165073,-0.056115,0.327610,0.052624,0.286369,-0.694977,-0.496204,-1
12107,-0.226806,-0.071491,-0.103074,0.058024,0.027885,0.230971,0.353388,0.222439,0.202664,0.123746,...,0.039351,-0.026419,-0.088226,0.044968,0.304888,0.470597,0.619008,-0.003580,-0.444521,-1


In [105]:
from sklearn.model_selection import train_test_split

label = vec_data['label']
attrs = vec_data.drop(columns=['label'])

X_train, X_test, y_train, y_test = train_test_split(
        attrs, label, test_size=0.25, random_state=1000)

In [106]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

Accuracy: 0.5535006605019815


In [107]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=30, random_state=0)
clf.fit(X_train, y_train)

score = clf.score(X_test, y_test)
print("Accuracy:", score)

Accuracy: 0.5211360634081902
